In [36]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from random import randint
import  random
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from cmath import nan
import urllib
import urllib.request
import re
import time
driver_path = '/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2/chromedriver'


In [37]:
url = 'https://sportsbook.draftkings.com/leagues/mma/ufc'

# load driver
driver = webdriver.Chrome(executable_path=driver_path)
driver.get(url)
time.sleep(5)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# get all tables
tables = soup.find_all('table')
# open first table
t1 = pd.read_html(str(tables[0]))[0]

# grab all links from table 1
links = []
for link in tables[0].find_all('a'):
    links.append(link.get('href'))


In [38]:
# find non-sgp odds
# drop all items in list with sgpmode in it
links = [x for x in links if 'sgpmode' not in x]
#links to df
links_df = pd.DataFrame(links, columns=['links'])
# reindex
links_df.reset_index(drop=True, inplace=True)

In [40]:
# add links from links_df to the main df by index
merged_df = pd.merge(t1, links_df, left_index=True, right_index=True)
# get first column name
col_name = merged_df.columns[0]
# change it to 'Date - Fighter'
merged_df.rename(columns={col_name:'Date - Fighter'}, inplace=True)

# Split the 'Date - Fighter' column into two columns at 'PM'
merged_df[['Time', 'Fighter']] = merged_df['Date - Fighter'].str.split('PM', expand=True)
# Drop the 'Date - Fighter' column
merged_df.drop(columns=['Date - Fighter'], inplace=True)
# drop the Point Spread column
merged_df.drop(columns=['Point Spread'], inplace=True)
# New column for Rounds
merged_df['Rounds'] = merged_df['Total Rounds'].str[0:5]
# New column for Round Odds
merged_df['Round Odds'] = merged_df['Total Rounds'].str[5:]
# drop the Total Rounds column
merged_df.drop(columns=['Total Rounds'], inplace=True)
# Move Moneyline and links to right side of df
cols_at_end = ['Moneyline', 'links']
merged_df = merged_df[[c for c in merged_df if c not in cols_at_end]
        + [c for c in cols_at_end if c in merged_df]]

# add the 'https://www.sportsbook.draftkings.com' to the links
merged_df['links'] = 'https://www.sportsbook.draftkings.com' + merged_df['links']

merged_df

,Time,Fighter,Rounds,Round Odds,Moneyline,links
0,3:45,Karine Silva,O 2.5,−130,−155,https://www.sportsbook.draftkings.com/event/ka...
1,3:45,Maryna Moroz,U 2.5,+100,+130,https://www.sportsbook.draftkings.com/event/ka...
2,4:15,Andrea Lee,O 2.5,−210,+260,https://www.sportsbook.draftkings.com/event/an...
3,4:15,Natalia Silva,U 2.5,+160,−325,https://www.sportsbook.draftkings.com/event/an...
4,4:45,Andre Petroski,O 1.5,−130,−225,https://www.sportsbook.draftkings.com/event/an...
5,4:45,Gerald Meerschaert,U 1.5,+100,+185,https://www.sportsbook.draftkings.com/event/an...
6,5:15,Brad Katona,O 2.5,−230,−175,https://www.sportsbook.draftkings.com/event/br...
7,5:15,Cody Gibson,U 2.5,+175,+145,https://www.sportsbook.draftkings.com/event/br...
8,5:45,Austin Hubbard,O 2.5,−166,−170,https://www.sportsbook.draftkings.com/event/au...
9,5:45,Kurt Holobaugh,U 2.5,+130,+142,https://www.sportsbook.draftkings.com/event/au...


# Scrape Sub-pages

In [44]:
url = "https://sportsbook.draftkings.com/event/aljamain-sterling-vs-sean-o'malley/28861392?category=odds&subcategory=fight-lines"
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
div = soup.find('div', {'class': 'sportsbook-event-accordion__wrapper expanded'})
# print div pretty
print(div.prettify())

<div class="sportsbook-responsive-card-container__body">
 <div aria-labelledby="game_category_Odds" class="sportsbook-responsive-card-container__card selected">
  <div class="sportsbook-tabbed-subheader" data-testid="sportsbook-tabbed-subheader">
   <button class="side-arrow side-arrow--right" data-testid="button-side-arrow" type="button">
    <svg class="side-arrow-icon" data-testid="side-arrow-right" fill="none" height="26" viewbox="0 0 11 26" width="11" xmlns="http://www.w3.org/2000/svg">
     <path d="M1 25L9 13L1 1" stroke="#fff" stroke-width="2">
     </path>
    </svg>
   </button>
   <div class="sportsbook-tabbed-subheader__inner">
    <div aria-label="Subcategories" class="sportsbook-tabbed-subheader__tabs" role="tablist">
     <a aria-selected="false" class="sportsbook-tabbed-subheader__tab-link" data-testid="sportsbook-tabbed-subheader-tab-link" href="/event/aljamain-sterling-vs-sean-o'malley/28861392?category=odds&amp;subcategory=popular" id="subcategory_Popular" role="tab"